In [1]:
import pandas as pd
import json
import requests
import datetime
import time
import urllib.parse
import unicodedata
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
spotify_token = "BQC08JJy93VZj9ZE9ksfB7GAO3CD2NDeX1feic47EaDn5wH_1ic4dhJ_4XT4IsLgj2qsD166e-ED7c6pBMEw1tKA-aC9I5jXMrXO6gyrhqcTa8QTTnA46DOks7SriUQtV7JgcVRhEMJ69ZYwvVAsr4sF1GMVvHBbNsMR2sr39HQ3GqNmTj_1oUsT-DUjiN9A2GDGjT4U7vAbtZm5aREThFGx9_ptPjCANoL70cyVR4c"
genius_token = "WJrlisPS9YMSVBPq9yVSMR1HJmzw1kBdgjAAPo3HTjXGgEM2NpglSCjFxA29lb_D"

In [3]:
start_time = datetime.datetime.now()

In [4]:
dates = []
starting_date = datetime.datetime.strptime("2023-01-05", "%Y-%m-%d")
ending_date = datetime.datetime.strptime("2024-12-26", "%Y-%m-%d")
while starting_date <= ending_date:
    dates.append(starting_date.strftime("%Y-%m-%d"))
    starting_date = starting_date + datetime.timedelta(days=7)

dates


['2023-01-05',
 '2023-01-12',
 '2023-01-19',
 '2023-01-26',
 '2023-02-02',
 '2023-02-09',
 '2023-02-16',
 '2023-02-23',
 '2023-03-02',
 '2023-03-09',
 '2023-03-16',
 '2023-03-23',
 '2023-03-30',
 '2023-04-06',
 '2023-04-13',
 '2023-04-20',
 '2023-04-27',
 '2023-05-04',
 '2023-05-11',
 '2023-05-18',
 '2023-05-25',
 '2023-06-01',
 '2023-06-08',
 '2023-06-15',
 '2023-06-22',
 '2023-06-29',
 '2023-07-06',
 '2023-07-13',
 '2023-07-20',
 '2023-07-27',
 '2023-08-03',
 '2023-08-10',
 '2023-08-17',
 '2023-08-24',
 '2023-08-31',
 '2023-09-07',
 '2023-09-14',
 '2023-09-21',
 '2023-09-28',
 '2023-10-05',
 '2023-10-12',
 '2023-10-19',
 '2023-10-26',
 '2023-11-02',
 '2023-11-09',
 '2023-11-16',
 '2023-11-23',
 '2023-11-30',
 '2023-12-07',
 '2023-12-14',
 '2023-12-21',
 '2023-12-28',
 '2024-01-04',
 '2024-01-11',
 '2024-01-18',
 '2024-01-25',
 '2024-02-01',
 '2024-02-08',
 '2024-02-15',
 '2024-02-22',
 '2024-02-29',
 '2024-03-07',
 '2024-03-14',
 '2024-03-21',
 '2024-03-28',
 '2024-04-04',
 '2024-04-

In [5]:
def try_request(method, url, headers):
    error = True
    i = 0
    while error:
        try:
            response = requests.request(method, url, headers=headers)
            if response.status_code != 200 and response.status_code != 404:
                print(response)
                raise Exception("Retry")
            error = False
        except:
            if i > 10:
                raise Exception("Too many retries for request: " + url)
            time.sleep(1.0 + (i*5))
            i += 1
            print("Retrying " + str(i) +  ":\t" + url)
    return response
            

In [6]:
def normalize_string(s):
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("utf-8")
    s = re.sub(r'[^a-zA-Z ]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()

    return s

In [7]:
def load_countries():
    with open('countries.json') as f:

        json_countries = json.load(f)
        map(lambda y: y['code'], json_countries)

        result = [x['code'] for x in json_countries]
        return result

countries = ["GLOBAL", "IT", "SE", "US", "GB", "DK", "FR", "DE", "ES"] 


#load_countries()

In [8]:
def download_chart_for_country_and_date(country, date):
    time.sleep(0.01) 
    url = f'https://charts-spotify-com-service.spotify.com/auth/v0/charts/regional-{country}-weekly/{date}'
    headers = {
      'accept': 'application/json',
      'accept-language': 'en-US,en;q=0.9',
      'app-platform': 'Browser',
      'authorization': f"Bearer {spotify_token}",
      'cache-control': 'no-cache',
      'content-type': 'application/json',
      'origin': 'https://charts.spotify.com',
      'pragma': 'no-cache',
      'priority': 'u=1, i',
      'referer': 'https://charts.spotify.com/',
      'sec-ch-ua': '"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"macOS"',
      'sec-fetch-dest': 'empty',
      'sec-fetch-mode': 'cors',
      'sec-fetch-site': 'same-site',
      'spotify-app-version': '0.0.0.production',
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
    }

    response = try_request("GET", url, headers=headers)
    # print(country, date)

    if 'entries' not in response.json():
        return pd.DataFrame([])

    songs = []
    songs_basic = []
    for song in response.json()['entries']:
        if song['missingRequiredFields'] and song['trackMetadata']['artists'] == []:
            print("Skipping song due to missing artists date: " + str(date) + " country: " + country)
            print(song)
            continue
        new_song = {
            'spotifyId': song['trackMetadata']['trackUri'],
            'trackName': song['trackMetadata']['trackName'],
            'artistName': song['trackMetadata']['artists'][0]['name'],
            'releaseDate': song['trackMetadata']['releaseDate'],
            'currentRank': song['chartEntryData']['currentRank'],
            'peakRank': song['chartEntryData']['peakRank'],
            'weeksOnChart': song['chartEntryData']['appearancesOnChart'],
            'numStreams':song['chartEntryData']['rankingMetric']['value'],
            'entryDate': song['chartEntryData']['entryDate']
        }

        new_song_basic = {
            'spotifyId': song['trackMetadata']['trackUri'],
            'trackName': song['trackMetadata']['trackName'],
            'artistName': song['trackMetadata']['artists'][0]['name'],
            'releaseDate': song['trackMetadata']['releaseDate']
        }
        songs.append(new_song)
        songs_basic.append(new_song_basic)

    df = pd.json_normalize(songs)
    df = df.assign(Country=country)
    df = df.assign(Week=response.json()['displayChart']['date'])
    df = df.set_index(['Country', 'Week', 'spotifyId'])

    df_tracks = pd.json_normalize(songs_basic)

    return df, df_tracks

In [9]:
def download_all_charts_for_date(date):
    df = pd.DataFrame([])
    df_tracks = pd.DataFrame([])
    for country in countries:
        chart, chart_basic = download_chart_for_country_and_date(country, date)

        df = pd.concat([df, chart])
        df_tracks = pd.concat([df_tracks, chart_basic])
    df = df.reset_index()
    df_tracks = df_tracks.reset_index()
    return df, df_tracks

In [10]:
df = pd.DataFrame([])
df_tracks = pd.DataFrame([])
try:
    chart_info = json.loads("charting_data2024-05-09.json")
    print("Loading previous charting data")
except:
    print("Creating new charting data")
    chart_info = {}




Creating new charting data


In [11]:
for i_date in tqdm(dates):
    
    try:
        if i_date in chart_info:
            chart, chart_basic = chart_info[i_date]
            chart = pandas.read_json(chart)
            chart_basic = pandas.read_json(chart_basic)
        else:
            chart, chart_basic = download_all_charts_for_date(i_date)
            chart_info[i_date] = (chart.to_json(), chart_basic.to_json())
        
        df = pd.concat([df, chart])
        df_tracks = pd.concat([df_tracks, chart_basic])
    except Exception as e:
        with open("charting_data" + i_date + ".json" , "w" ) as write:
            json.dump( chart_info , write )
            print("Dumping into " + "charting_data" + i_date + ".json")
            raise e
        


df_tracks = df_tracks.assign(geniusId=None)
df_tracks
# chart = download_all_charts_for_date('2024-12-05')
# chart


100%|███████████████████████████████████████████████████████████████████████████████████████████| 104/104 [03:43<00:00,  2.15s/it]


,index,spotifyId,trackName,artistName,releaseDate,geniusId
0,0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,None
1,1,spotify:track:3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith,2022-09-22,None
2,2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,As It Was,Harry Styles,2022-05-20,None
3,3,spotify:track:4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),David Guetta,2022-08-26,None
4,4,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2022-12-02,None
...,...,...,...,...,...,...
1795,195,spotify:track:4gdpQ6zuDsePnD1YIDJCRw,Potra Salvaje - Hard Remix,Isabel Aaiún,2023-10-14,None
1796,196,spotify:track:5humEOJehpAwMJEJmb0JJ2,The Christmas Song (Merry Christmas To You),Nat King Cole,1961-09-25,None
1797,197,spotify:track:3jWGIzAGXwaTEP6fX4fAEd,El Burrito Sabanero,Elvis Crespo,2023-11-13,None
1798,198,spotify:track:6jfp0rIQSdBoijRTR7TC9l,TRAQUETO,iZaak,2024-07-12,None


In [12]:
df_tracks = df_tracks.drop_duplicates(subset='spotifyId', keep='first')

df_tracks

,index,spotifyId,trackName,artistName,releaseDate,geniusId
0,0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,None
1,1,spotify:track:3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith,2022-09-22,None
2,2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,As It Was,Harry Styles,2022-05-20,None
3,3,spotify:track:4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),David Guetta,2022-08-26,None
4,4,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2022-12-02,None
...,...,...,...,...,...,...
1783,183,spotify:track:5B7T5rlFMewr6Npbuw8q8t,En Navidad - En directo,Rosana,,None
1788,188,spotify:track:4ApLAqNEdqMwgF3Y8MRSXj,7 Trompetas,Cris Mj,2024-12-19,None
1790,190,spotify:track:30QvxRsCWIuH6MhAcpxDc6,Infrarrojo / Ultravioleta,Hoke,2024-12-18,None
1794,194,spotify:track:74GFH65pkGCmjo3qxmmkE6,Navidad Junto A Ti,David Bisbal,2024-11-15,None


In [13]:
def find_genius_id(track_name, track_main_artist):
    track_name = re.sub(r"\s*\[.*?\]", "", track_name)

    if "feat." in track_name.lower() or "with " in track_name.lower() or "from " in track_name.lower():
        track_name = re.sub(r"\s*\(.*?\)", "", track_name)

    time.sleep(0.010)
    query = {
        "access_token": genius_token,
        "q": f"{track_name}, {track_main_artist}"
    }
    query = urllib.parse.urlencode(query)
    #print(query)
    url = f'https://api.genius.com/search?{query}'
    payload = {}
    headers = {
      'Accept': 'application/json, text/plain, */*',
      'Sec-Fetch-Site': 'same-site',
      'Origin': 'https://docs.genius.com',
      'Sec-Fetch-Dest': 'empty',
      'Accept-Language': 'en-GB,en;q=0.9',
      'If-None-Match': 'W/"2a87e230fcad2086285f6b15cfc4ff90"',
      'Sec-Fetch-Mode': 'cors',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.2 Safari/605.1.15',
      'Accept-Encoding': 'gzip, deflate, br',
      'Referer': 'https://docs.genius.com/',
      'Priority': 'u=3, i'
    }

    response = try_request("GET", url, headers=headers)

    response_json = response.json()

    hits = response_json.get('response', {}).get('hits', [])
    hits = [x for x in hits if normalize_string(track_main_artist.lower()) in normalize_string(x.get('result').get('artist_names').lower())]

    if not hits:
        if "(" in track_name:
            stripped_track_name = re.sub(r"\s*\(.*?\)", "", track_name)
            return find_genius_id(stripped_track_name, track_main_artist)
        elif "-" in track_name:
            stripped_track_name = re.sub(r"\s*-\s*.*", "", track_name)
            return find_genius_id(stripped_track_name, track_main_artist)
        elif re.search(r'[^a-zA-Z0-9 ]', track_name):
            stripped_track_name = re.sub(r'[^a-zA-Z ]', ' ', track_name)
            stripped_track_name = re.sub(r'\s+', ' ', stripped_track_name).strip()
            return find_genius_id(stripped_track_name, track_main_artist)
        else:
            return None  # Return None if no results

    track_response = hits[0]['result']

    release_date_components = track_response.get('release_date_components', {})
    if release_date_components:
        release_date = datetime.datetime(
            release_date_components.get('year', 1) or 1,
            release_date_components.get('month', 1) or 1,
            release_date_components.get('day', 1) or 1
        )
        formatted_release_date = release_date.strftime("%Y-%m-%d")
    else:
        formatted_release_date = None

    return {
        'geniusId': track_response.get('id'),
        'geniusTrackName': track_response.get('title'),
        'geniusArtistName': track_response.get('primary_artist_names'),
        'geniusReleaseDate': formatted_release_date
    }


In [14]:
for index, row in tqdm(df_tracks[df_tracks['geniusId'].isna()].iterrows()):
    spotify_id = row.get('spotifyId')
    genius_data = find_genius_id(row["trackName"], row["artistName"])
    if not genius_data:
        continue
    df_tracks.loc[df_tracks["spotifyId"] == spotify_id, ["geniusId", "geniusTrackName", "geniusArtistName", "geniusReleaseDate"]] = [
        genius_data['geniusId'], genius_data['geniusTrackName'], genius_data['geniusArtistName'], genius_data['geniusReleaseDate']
    ]
# find_genius_id("die with a smile", "lady gaga")
df_tracks["geniusId"] = df_tracks["geniusId"].astype("Int64")
df_tracks

12194it [2:35:17,  1.31it/s]
/var/folders/w3/ppd716f554v907_mxnlt679m0000gn/T/ipykernel_38955/4002356097.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tracks["geniusId"] = df_tracks["geniusId"].astype("Int64")


,index,spotifyId,trackName,artistName,releaseDate,geniusId,geniusTrackName,geniusArtistName,geniusReleaseDate
0,0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09
1,1,spotify:track:3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith,2022-09-22,8302862,Unholy,Sam Smith & Kim Petras,2022-09-22
2,2,spotify:track:4Dvkj6JhhA12EX05fT7y2e,As It Was,Harry Styles,2022-05-20,7844309,As It Was,Harry Styles,2022-04-01
3,3,spotify:track:4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),David Guetta,2022-08-26,3037193,I’m Good (Blue),David Guetta & Bebe Rexha,2022-08-26
4,4,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2022-12-02,8601823,Creepin’,"Metro Boomin, The Weeknd & 21 Savage",2022-12-02
...,...,...,...,...,...,...,...,...,...
1783,183,spotify:track:5B7T5rlFMewr6Npbuw8q8t,En Navidad - En directo,Rosana,,1846641,En Navidad,Rosana,1998-01-01
1788,188,spotify:track:4ApLAqNEdqMwgF3Y8MRSXj,7 Trompetas,Cris Mj,2024-12-19,11054491,7 Trompetas,Cris Mj,2024-12-19
1790,190,spotify:track:30QvxRsCWIuH6MhAcpxDc6,Infrarrojo / Ultravioleta,Hoke,2024-12-18,11232623,Infrarrojo / Ultravioleta,HOKE,2024-12-18
1794,194,spotify:track:74GFH65pkGCmjo3qxmmkE6,Navidad Junto A Ti,David Bisbal,2024-11-15,11215578,Navidad Junto A Ti,David Bisbal,None


In [15]:
def get_genius_song_info(genius_id):
    time.sleep(0.010)
    query = {
        "access_token": genius_token,
    }
    query = urllib.parse.urlencode(query)

    url = f'https://api.genius.com/songs/{genius_id}?{query}'
    payload = {}
    headers = {
      'Accept': 'application/json, text/plain, */*',
      'Sec-Fetch-Site': 'same-site',
      'Origin': 'https://docs.genius.com',
      'Sec-Fetch-Dest': 'empty',
      'Accept-Language': 'en-GB,en;q=0.9',
      'If-None-Match': 'W/"2a87e230fcad2086285f6b15cfc4ff90"',
      'Sec-Fetch-Mode': 'cors',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.2 Safari/605.1.15',
      'Accept-Encoding': 'gzip, deflate, br',
      'Referer': 'https://docs.genius.com/',
      'Priority': 'u=3, i'
    }

    response = try_request("GET", url, headers=headers)
    response_json = response.json()

    song = response_json.get('response', {}).get('song', {})
    song_relationships = song.get('song_relationships', [])
    song_relationships = [x for x in song_relationships if x.get('relationship_type', None) in {'samples', 'interpolates'} and len(x.get('songs', [])) > 0]
    song_image = [{'id': genius_id, 'type': 'song', 'imageURL': song.get('song_art_image_url')}]
    
    producer_images = [{'id': x.get('id'),'type': 'artist', 'imageURL': x.get('image_url')} for x in song.get('producer_artists')]
    writer_images = [{'id': x.get('id'),'type': 'artist', 'imageURL': x.get('image_url')} for x in song.get('writer_artists')]
    artist_images = [{'id': x.get('id'),'type': 'artist', 'imageURL': x.get('image_url')} for x in song.get('primary_artists')]
    artist_images = artist_images + [{'id': x.get('id'),'type': 'artist', 'imageURL': x.get('image_url')} for x in song.get('featured_artists')]
    

    genius_data = {
        'song_language': song.get('language', None),
        'song_relationships': song_relationships,
        'primary_artists':[{'geniusId': genius_id, 'type': 'primary', 'artistId': x.get('id'), 'name': x.get('name')} for x in song.get('primary_artists')],
        'producers': [{'geniusId': genius_id, 'type': 'producer', 'artistId': x.get('id'), 'name': x.get('name')} for x in song.get('producer_artists')],
        'writers': [{'geniusId': genius_id, 'type': 'writer', 'artistId': x.get('id'), 'name': x.get('name')} for x in song.get('writer_artists')],
        'images' : producer_images + writer_images + artist_images + song_image,
        'featured_artists' : [{'geniusId': genius_id, 'type': 'feature', 'artistId': x.get('id'), 'name': x.get('name')} for x in song.get('featured_artists')]
    }

    return genius_data

def get_sampling_data(genius_id, genius_song_info):
    new_songs = []
    relationships = []
    # samples
    for samples in genius_song_info.get('song_relationships'):
        for songs in samples.get('songs'):

            genius_track = df_tracks[df_tracks["geniusId"] == genius_id]
            if genius_track["geniusArtistName"].iloc[0] == songs.get('artist_names') and genius_track["geniusTrackName"].iloc[0] in songs.get('title'):
                #print(f"skipping {songs.get('artist_names')} - {songs.get('title')} because of self-reference")
                continue

            release_date_components = songs.get('release_date_components', {})
            if release_date_components:
                release_date = datetime.datetime(
                    release_date_components.get('year', 1) or 1,
                    release_date_components.get('month', 1) or 1,
                    release_date_components.get('day', 1) or 1
                )
                formatted_release_date = release_date.strftime("%Y-%m-%d")
            else:
                formatted_release_date = None
            # print(genius_song_info.get('song_relationships'))
            x = {
                "geniusId": songs.get('id'),
                "geniusTrackName": songs.get('title'),
                "geniusArtistName": songs.get('artist_names'),
                "geniusReleaseDate": formatted_release_date,
            }

            new_songs.append(x)
            relationships.append({
                'from_genius_id': genius_id,
                'to_genius_id': songs.get('id'),
                'type': samples.get('relationship_type')
            })

    df_new_songs = pd.json_normalize(new_songs)
    df_relationships = pd.json_normalize(relationships)
    return df_new_songs, df_relationships

df_relationships = pd.DataFrame([])
df_contributions = pd.DataFrame([])
song_languages = []
images = []
for genius_id in tqdm(df_tracks[df_tracks["geniusId"].notna()]["geniusId"].tolist()):
    genius_song_info = get_genius_song_info(genius_id)
    ############################################################# NOTE: song relations do not yeild languages
    df_new_songs, df_new_relationships = get_sampling_data(genius_id, genius_song_info)
    df_tracks = pd.concat([df_tracks, df_new_songs])
    df_relationships = pd.concat([df_relationships, df_new_relationships])
    df_contributions = pd.concat([df_contributions, pd.json_normalize(genius_song_info['producers']), pd.json_normalize(genius_song_info['writers']), pd.json_normalize(genius_song_info['featured_artists']), pd.json_normalize(genius_song_info['primary_artists'])])
    song_languages.append({'geniusId': genius_id, 'trackLanguage':genius_song_info['song_language']})
    images.extend(genius_song_info['images'])
    
df_languages = pd.DataFrame(song_languages)
df_tracks = df_tracks.merge(df_languages, on='geniusId', how='left')
df_images = pd.DataFrame(images)
df_tracks

  0%|▏                                                                                       | 17/11857 [00:31<5:07:38,  1.56s/it]

<Response [503]>
Retrying 1:	https://api.genius.com/songs/239378?access_token=WJrlisPS9YMSVBPq9yVSMR1HJmzw1kBdgjAAPo3HTjXGgEM2NpglSCjFxA29lb_D


100%|█████████████████████████████████████████████████████████████████████████████████████| 11857/11857 [3:04:22<00:00,  1.07it/s]


,index,spotifyId,trackName,artistName,releaseDate,geniusId,geniusTrackName,geniusArtistName,geniusReleaseDate,trackLanguage
0,0.0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09,en
1,0.0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09,en
2,1.0,spotify:track:3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith,2022-09-22,8302862,Unholy,Sam Smith & Kim Petras,2022-09-22,en
3,1.0,spotify:track:3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith,2022-09-22,8302862,Unholy,Sam Smith & Kim Petras,2022-09-22,en
4,2.0,spotify:track:4Dvkj6JhhA12EX05fT7y2e,As It Was,Harry Styles,2022-05-20,7844309,As It Was,Harry Styles,2022-04-01,en
...,...,...,...,...,...,...,...,...,...,...
20058,NaN,NaN,NaN,NaN,NaN,7010519,Kapuze im Club,"RAF Camora, 187 Strassenbande, Gzuz, Maxwell 1...",2021-07-16,NaN
20059,NaN,NaN,NaN,NaN,NaN,6492688,Verpennt,"187 Strassenbande, Bonez MC, LX & Sa4",2021-03-26,NaN
20060,NaN,NaN,NaN,NaN,NaN,1240631,Never Stop,Planetshakers,2007-01-01,NaN
20061,NaN,NaN,NaN,NaN,NaN,677353,Auf der Jagd,Celo & Abdi,2010-12-23,NaN


In [16]:
df.to_csv("output.csv", encoding='utf-8', index=True, header=True)
df_tracks.to_csv("output_tracks.csv", encoding='utf-8', index=False, header=True)
df_relationships.to_csv("output_relationships.csv", encoding='utf-8', index=False, header=True)
df_images = df_images.drop_duplicates(subset=['id'])
df_images.to_csv("image_urls.csv", encoding='utf-8', index=False, header=True)

In [17]:
df_merged = df_tracks[df_tracks['spotifyId'].notna()].merge(df_relationships, left_on="geniusId", right_on="from_genius_id", how="inner")
df_final = df_merged.merge(df_tracks[df_tracks['spotifyId'].isna()], left_on="to_genius_id", right_on="geniusId", how="inner", suffixes=("_from", "_to"))
df_final = df_final.drop_duplicates()
df_final.to_csv("output_merged.csv", encoding='utf-8', index=False, header=True)

In [18]:
df_contributions
df_merged_contributions = df_tracks[df_tracks['spotifyId'].notna()].merge(df_contributions, left_on="geniusId", right_on="geniusId", how="inner")
df_merged_contributions
# df_merged_contributions.to_csv("output_merged_contributions.csv", encoding='utf-8', index=False, header=True)

,index,spotifyId,trackName,artistName,releaseDate,geniusId,geniusTrackName,geniusArtistName,geniusReleaseDate,trackLanguage,type,artistId,name
0,0.0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09,en,producer,182843,Carter Lang
1,0.0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09,en,producer,1038467,Rob Bisel
2,0.0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09,en,writer,41579,SZA
3,0.0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09,en,writer,1038467,Rob Bisel
4,0.0,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,2022-12-08,8616820,Kill Bill,SZA,2022-12-09,en,writer,182843,Carter Lang
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033438,190.0,spotify:track:30QvxRsCWIuH6MhAcpxDc6,Infrarrojo / Ultravioleta,Hoke,2024-12-18,11232623,Infrarrojo / Ultravioleta,HOKE,2024-12-18,es,producer,1412426,PMP
1033439,190.0,spotify:track:30QvxRsCWIuH6MhAcpxDc6,Infrarrojo / Ultravioleta,Hoke,2024-12-18,11232623,Infrarrojo / Ultravioleta,HOKE,2024-12-18,es,writer,1395600,HOKE
1033440,190.0,spotify:track:30QvxRsCWIuH6MhAcpxDc6,Infrarrojo / Ultravioleta,Hoke,2024-12-18,11232623,Infrarrojo / Ultravioleta,HOKE,2024-12-18,es,primary,1395600,HOKE
1033441,194.0,spotify:track:74GFH65pkGCmjo3qxmmkE6,Navidad Junto A Ti,David Bisbal,2024-11-15,11215578,Navidad Junto A Ti,David Bisbal,None,es,primary,344442,David Bisbal


In [19]:
df_contributions.to_csv("output_contributions.csv", encoding='utf-8', index=False, header=True)